In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import sys
sys.path.insert(0, '..//')

In [24]:
import pandas as pd
from pathlib import Path
from src.models import report

# Running Recharging Tenants

This project allows for site submeters to be recharged for their energy usage. This is done by calculating a 'recharge rate' based on the monthly energy charge for the submeters connected main meter. 

### Data Imports

In [25]:
data_path = Path(r'..//data/example_data')
results_path = Path(r'..//data/example_results')

""" Retrieve the current invoice data for the gas, electricity and water."""

gas_path = data_path / 'example_gas_invoice.csv'
elec_2_path = data_path / 'example_electric_invoice_2.csv'
water_path = data_path / 'example_water_invoice.csv'

recharge_readings_path = data_path / 'example_tenant_readings.csv'

""" Folder where you want the resulting recharging csv files to be saved to."""

save_folder = results_path

""" Where to recover the historical charges and readings from and save to. \
  This data can be used in future if meters fail or for historical analysis/forecasting."""

historical_charges_path = results_path / 'Test Site_MARCH_2023/historical_charges.csv'
historical_readings_path = results_path / 'Test Site_MARCH_2023/historical_readings.csv'

## Meter readings

`recharge_readings_path` is the csv holding the tenant meter reads. Lets have a look and go through them.

In [26]:
pd.read_csv(recharge_readings_path, index_col=0)

,Datetime,Site,Utility/Meter,Sub Utility,Flow,Previous meter reading,Previous meter reading date,Present meter reading,Present meter reading date
Unnamed: 0,,,,,,,,,
0,3/1/23,16,E,0,False,1.915430e+05,3/9/23,1.925770e+05,4/6/23
1,3/1/23,16,G,0,False,2.276800e+04,3/9/23,2.310300e+04,4/6/23
2,3/1/23,16,W,0,False,1.699370e+02,3/9/23,1.718460e+02,4/6/23
3,3/1/23,4,E,0,False,1.346280e+05,3/9/23,1.362850e+05,4/6/23
4,3/1/23,4,G,0,False,7.980965e+04,3/9/23,8.132735e+04,4/6/23
5,3/1/23,4,W,0,False,7.628600e+03,3/9/23,7.628600e+03,4/6/23
6,3/1/23,5,E,0,False,3.583410e+05,3/9/23,3.602850e+05,4/6/23
7,3/1/23,5,G,0,False,1.489816e+05,3/9/23,1.503911e+05,4/6/23
8,3/1/23,5,W,0,False,1.287687e+04,3/9/23,1.292468e+04,4/6/23


- `Site` - The name of the tenant can be a string or int. It must match the value used in the additional attributes you'll find below.

- `Utility/Meter` - What type of utility is the reading you're providing this is either E, G or W for electricity, gas and water.

- `Sub Utility` - Some tenants may have multiple meters per utility for their sites. Here you can give a sub utility name for the different meters. If only one meter per utility set the value to `0`.

- `Flow` - A tenant may be a system that converts energy and returns supply as a different utility that you want excluded from their energy bill. For example, they could consume gas to generate domestic hot water. In this situation you'd have two gas meters, the incoming gas meter would have a `Flow` of `False` and the meter that returns the hot water would have a `True` value for `Flow`. This subtracts the energy generated by the tenant from the energy they consumed.

An example of this is in the above dataframe in rows 25 & 26 for `Site` 15.

After the first usage of recharging tenants, the program generates a `new_form.csv` that saves all the above information for the next use meaning the only value users will need to input will be `Present meter reading` & `Present meter reading date`.


<b> Note: </b> If you do not have historical data csvs. You can create blank ones with just the column headers found in the examples in `example_results`.

## Additional attributes

Descriptions of the different attributes are provided below.

- `id_mappings` - A dictionary of dictionaries. This connects the site tenants to the main meters for their utilities using `mpan` for electricity, `mpr` for gas and `water` for water. This is used to connect the recharging rate of the specific main meter to the tenants connected to that meter.

- `fixed_rate_mapping` - A dictionary of dictionaries. Some sites may want to apply a fixed charge to tenants for energy usage through their main meters. Here you can apply any fixed rate values based on the utility types. Don't list the tenant or an individual tenant if you don't want to set a fixed charge.

- `commercial_list` - A list of tenant IDs. This seperates commercial and residential tenants. This needs to be done to ensure the correct VAT rate is applied to the tenant and splits the tenants into a `comercial_charges.csv` & `resident_charges.csv`. Any ID not listed is considered residential and the 20% VAT rate is applied.

- `readings_multiplier` - A dictionary of dictionaries. A sites meters may not be measuring kWh. If they use something different such as cubic meters (Sm3), cubic feet (cf) or hundreds of cubic feet (hcf). This needs to be specified here as the recharge rate is calculated as GBP/kWh.

In [27]:
""" Mapping of the invoice MPAN, MPR and Water to the various tenants. """
# MPAN = Electricity | MPR = Gas | Water = Water
id_mappings = {'1': {'mpan': '1098', 'mpr': '5678', 'water': 'abcd'},
            '2': {'mpan': '1098', 'mpr': '5678', 'water': 'abcd'},
            '3': {'mpan': '1098', 'mpr': '5678', 'water': 'abcd'},
            'House': {'mpan': '1098', 'water': 'abcd'},
            '4': {'mpan': '1098', 'mpr': '7890', 'water': 'abcd'},
            '5': {'mpan': '1098', 'mpr': '5678', 'water': 'abcd'},
            '6': {'mpan': '1098', 'water': 'abcd'},
            '7': {'mpan': '1098', 'water': 'abcd'},
            '8': {'mpan': '1098', 'water': 'abcd'},
            '9': {'mpan': '1098', 'water': 'abcd'},
            '10': {'mpan': '1098', 'mpr': '7890', 'water': 'abcd'},
            '11': {'mpan': '1098', 'water': 'abcd'},
            '12': {'mpan': '1098', 'water': 'abcd'},
            '13': {'mpan': '1098', 'water': 'abcd'},
            "14": {'mpan': '1098', 'mpr': '7890', 'water': 'abcd'},
            "15": {'mpan': '1098', 'mpr': '7890', 'water': 'abcd'},
            '16': {'mpan': '9876', 'mpr': '1234', 'water': 'efgh'},
            }

""" Mapping of any fixed rate charges to the tenants. If blank, none are applied. """
fixed_rate_mappings = {'4': {'Electric': 90.18, 'Gas': 72.53, 'Water':70.58},
                       '5': {'Electric': 84.30, 'Gas': 72.53, 'Water':70.58},
                          '10': {'Electric': 124.49, 'Gas': 75.47, 'Water': 72.78},
                          '14': {'Electric': 196.82, 'Gas': 89.13},
                          '15': {'Gas': 72.78},
                          '6': {'Electric': 28.12, 'Water': 66.32},
                          '7': {'Electric': 95.09, 'Water': 62.83},
                          '8': {'Electric': 67.86, 'Water': 62.83},
                          '9': {'Electric': 90.18, 'Water': 70.58},
                          '12': {'Electric': 75.47},
                          '13': {'Electric': 75.47},
                          '11': {'Electric': 137.98, 'Water': 72.78},
                          '16': {'Electric': 75.03, 'Water': 67.27}}

""" List of all commercial tenants. Ensures 20% VAT is applied to the correct tenants. """
commercial_list = ['4', '5', '6', '7',
                   '8', '9', '10', '11',
                   '12', '13', '14', '15', '16']

""" Mapping of the meter reading conversion to kwh. """
sm3_to_kwh = (39.5*1.02264)/3.6
cf3_to_kwh = (39.5*1.02264*2.83)/3.6

""" Applies the conversion factors to the needed meters. """
readings_multiplier = {'5': {'G': sm3_to_kwh},
                      '10': {'G': 1333.33},
                      '14': {'G': cf3_to_kwh},
                      '15': {'G': 323},
                      '16': {'G': sm3_to_kwh},
                      '1': {'G': sm3_to_kwh},
                      '3': {'G': sm3_to_kwh},
                      '2': {'G': sm3_to_kwh, 'E': -1},
                      '4': {'G': sm3_to_kwh},
                      '9': {'E': 10}}

# Run the recharging function for the desired month

Once all the needed variables are set you are ready to run the recharging tenants function. This generates 5 different files.

1. `new_form.csv` - This is the new form for measuring the next months energy readings where `previous readings` are the currents month `present readings`.
2. `commercial_charges.csv` - A break down of the charges for commercial tenants which can be used as their energy invoices.
3. `residential_charges.csv` - Same as `commercial_charges.csv` but for residential tenants.
4. `historical_charges.csv` & `historical_readings.csv` - The charges and readings from all previous months updated to include the current months data as well.

In [28]:
""" --- Set up the site object --- """
rotherham_site = report.Site('Test Site',
                             id_mappings = id_mappings,
                             fixed_rate_mappings = fixed_rate_mappings,
                             readings_multiplier = readings_multiplier,
                             commercial_list = commercial_list,
                             reading_path = recharge_readings_path,
                             water_path=water_path,
                             gas_path=gas_path,
                             electric_path = elec_2_path,
                             historical_charges_path = historical_charges_path,
                             historical_readings_path = historical_readings_path,
                             save_folder = save_folder)

""" --- Create saving path & recharge the tenants --- """
date = pd.to_datetime('01/03/2023', format='%d/%m/%Y')
rotherham_site.create_saving_path(parent_folder=save_folder, recharging_date=date)
rotherham_site.recharging_tenants()

Recharging forms complete. Have a nice day.
